In [130]:
import pandas as pd
import numpy as np
from nltk import *
import difflib

In [131]:
df = pd.read_csv('good_reads_final.csv')
df.head()

,author_average_rating,author_gender,author_genres,author_id,author_name,author_page_url,author_rating_count,author_review_count,birthplace,book_average_rating,book_fullurl,book_id,book_title,genre_1,genre_2,num_ratings,num_reviews,pages,publish_date,score
0,4.010,female,"historical-fiction,",74489,Victoria Thompson\n,/author/show/74489.Victoria_Thompson,74399,6268,United States\n,4.020,https://www.goodreads.com/book/show/686717.Mur...,686717,\n Murder on St. Mark's Place\n,Mystery,Historical,5260,375,277,2000,3230
1,4.150,male,"literature-fiction,mystery-thrillers,",706255,Stieg Larsson\n,/author/show/706255.Stieg_Larsson,3726435,142704,Sweden\n,4.130,https://www.goodreads.com/book/show/2429135.Th...,2429135,\n The Girl with the Dragon Tattoo\n,Fiction,Mystery,2229163,65227,465,August 2005,3062
2,4.000,female,"romance,",5618190,Mimi Jean Pamfiloff\n,/author/show/5618190.Mimi_Jean_Pamfiloff,76496,7975,United States\n,3.990,https://www.goodreads.com/book/show/27833684-t...,27833684,\n Tailored for Trouble\n,Romance,Contemporary,2151,391,354,2016,4585
3,3.880,male,"fiction,memoir,",37871,José Donoso\n,/author/show/37871.Jos_Donoso,5522,489,Chile\n,4.140,https://www.goodreads.com/book/show/382975.The...,382975,\n The Obscene Bird of Night\n,Fiction,Magical Realism,1844,173,438,1970,1533
4,4.100,female,"young-adult,fantasy,",36122,Patricia C. Wrede\n,/author/show/36122.Patricia_C_Wrede,291013,13453,United States\n,4.010,https://www.goodreads.com/book/show/64207.Sorc...,64207,\n Sorcery & Cecelia: or The Enchanted Ch...,Fantasy,Young Adult,17051,1890,326,April 15th 1988,2105


In [132]:
df.columns

Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'author_rating_count',
       'author_review_count', 'birthplace', 'book_average_rating',
       'book_fullurl', 'book_id', 'book_title', 'genre_1', 'genre_2',
       'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'],
      dtype='object')

In [133]:
df = df[[ 'book_title', 'author_name', 'book_average_rating','num_ratings','author_average_rating',
         'author_rating_count' , 'genre_1', 'genre_2',  'publish_date', 'book_fullurl']]
df['book_title'] = df['book_title'].str.strip().str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22891 entries, 0 to 22890
Data columns (total 10 columns):
book_title               22891 non-null object
author_name              22891 non-null object
book_average_rating      22891 non-null float64
num_ratings              22891 non-null int64
author_average_rating    22891 non-null float64
author_rating_count      22891 non-null int64
genre_1                  22891 non-null object
genre_2                  22891 non-null object
publish_date             22610 non-null object
book_fullurl             22891 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 1.7+ MB


In [134]:
# Removing the rows with null items in publish date
df.dropna(subset=['publish_date'], inplace=True)
df.rename(columns={"publish_date": "publish_yr"}, inplace=True)

In [135]:
years = []
for dates in df['publish_yr'].str.split(' '):
    try:
        if len(dates) == 1:
            dates = int(dates[0])
        elif len(dates) == 2:
            dates = int(dates[1])
        elif len(dates) == 3:
            dates = int(dates[2])
        
            
    except Exception as e:
        dates = 0
        
        
    years.append(dates)
df['publish_yr'] = years

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22610 entries, 0 to 22890
Data columns (total 10 columns):
book_title               22610 non-null object
author_name              22610 non-null object
book_average_rating      22610 non-null float64
num_ratings              22610 non-null int64
author_average_rating    22610 non-null float64
author_rating_count      22610 non-null int64
genre_1                  22610 non-null object
genre_2                  22610 non-null object
publish_yr               22610 non-null int64
book_fullurl             22610 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 1.9+ MB


In [137]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

,book_average_rating,num_ratings,author_average_rating,author_rating_count,publish_yr
count,22610.000,22610.000,22610.000,22610.000,22610.000
mean,3.951,47094.957,3.961,173387.508,1949.534
std,0.289,181063.348,0.239,658157.904,311.349
min,1.850,1.000,1.820,6.000,-720.000
25%,3.770,849.000,3.810,4453.500,1998.000
50%,3.960,4491.500,3.970,25230.500,2011.000
75%,4.140,20451.500,4.120,112362.250,2013.000
max,5.000,3820921.000,5.000,21117318.000,2019.000


In [138]:
df = df[df['publish_yr'] >1980][df['author_rating_count'] > 4000][df['num_ratings']> 800]
df = df.drop(labels=['author_rating_count','num_ratings', 'publish_yr'], axis=1)

c:\users\sahaj oberoi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [139]:
df.head()

,book_title,author_name,book_average_rating,author_average_rating,genre_1,genre_2,book_fullurl
0,murder on st. mark's place,Victoria Thompson\n,4.020,4.010,Mystery,Historical,https://www.goodreads.com/book/show/686717.Mur...
1,the girl with the dragon tattoo,Stieg Larsson\n,4.130,4.150,Fiction,Mystery,https://www.goodreads.com/book/show/2429135.Th...
2,tailored for trouble,Mimi Jean Pamfiloff\n,3.990,4.000,Romance,Contemporary,https://www.goodreads.com/book/show/27833684-t...
4,sorcery & cecelia: or the enchanted chocolate pot,Patricia C. Wrede\n,4.010,4.100,Fantasy,Young Adult,https://www.goodreads.com/book/show/64207.Sorc...
5,"30 days of night, vol. 1",Steve Niles\n,3.800,3.770,Sequential Art,Sequential Art,https://www.goodreads.com/book/show/831829.30_...


In [140]:
def clean_text(df, cols):
    for col in cols:
        text =  []
        df[col] = df[col].str.strip()
        for item in df[col]:
            word = item.replace(" ", "")
            #print(word)
            text.append(word.lower())
        df[col] = text
clean_text(df , ['author_name', 'genre_1', 'genre_2']) # 'book_title',  
df.head()

,book_title,author_name,book_average_rating,author_average_rating,genre_1,genre_2,book_fullurl
0,murder on st. mark's place,victoriathompson,4.020,4.010,mystery,historical,https://www.goodreads.com/book/show/686717.Mur...
1,the girl with the dragon tattoo,stieglarsson,4.130,4.150,fiction,mystery,https://www.goodreads.com/book/show/2429135.Th...
2,tailored for trouble,mimijeanpamfiloff,3.990,4.000,romance,contemporary,https://www.goodreads.com/book/show/27833684-t...
4,sorcery & cecelia: or the enchanted chocolate pot,patriciac.wrede,4.010,4.100,fantasy,youngadult,https://www.goodreads.com/book/show/64207.Sorc...
5,"30 days of night, vol. 1",steveniles,3.800,3.770,sequentialart,sequentialart,https://www.goodreads.com/book/show/831829.30_...


In [141]:
def numbers_normalised(df, col_list):
    for col in col_list:
        maxx = df[col].max()
        minn = df[col].min()
        df[col] = round((df[col] - minn) / (maxx - minn), 3) 
    return df.head()

numbers_normalised(df, ['book_average_rating', 'author_average_rating'])

,book_title,author_name,book_average_rating,author_average_rating,genre_1,genre_2,book_fullurl
0,murder on st. mark's place,victoriathompson,0.649,0.621,mystery,historical,https://www.goodreads.com/book/show/686717.Mur...
1,the girl with the dragon tattoo,stieglarsson,0.697,0.701,fiction,mystery,https://www.goodreads.com/book/show/2429135.Th...
2,tailored for trouble,mimijeanpamfiloff,0.636,0.616,romance,contemporary,https://www.goodreads.com/book/show/27833684-t...
4,sorcery & cecelia: or the enchanted chocolate pot,patriciac.wrede,0.645,0.672,fantasy,youngadult,https://www.goodreads.com/book/show/64207.Sorc...
5,"30 days of night, vol. 1",steveniles,0.553,0.486,sequentialart,sequentialart,https://www.goodreads.com/book/show/831829.30_...


In [142]:
df = df[~df[['book_title', 'author_name']].duplicated()]

In [143]:
df['text'] = df['author_name'] + " " + df['genre_1'] + " " + df['genre_2'] + " " + df['book_title'].str.lower()
df['numerical'] = (1.5 * df['book_average_rating']) + df['author_average_rating']
final_df = df[['book_title', 'text', 'numerical']]
final_df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
final_df.head()

,book_title,text,numerical
0,murder on st. mark's place,victoriathompson mystery historical murder on ...,1.595
1,the girl with the dragon tattoo,stieglarsson fiction mystery the girl with the...,1.746
2,tailored for trouble,mimijeanpamfiloff romance contemporary tailore...,1.570
3,sorcery & cecelia: or the enchanted chocolate pot,patriciac.wrede fantasy youngadult sorcery & c...,1.639
4,"30 days of night, vol. 1",steveniles sequentialart sequentialart 30 days...,1.316


In [144]:
final_df['numerical'].sort_values(ascending=False).head()

3788   2.494
2454   2.428
4495   2.414
1147   2.406
2832   2.391
Name: numerical, dtype: float64

In [145]:
final_df.tail()

,book_title,text,numerical
8976,dark wolf rising,rhyannonbyrd fantasy romance dark wolf rising,1.462
8977,hidden moon,k.r.thompson youngadult fantasy hidden moon,1.503
8978,given to the pack,abbyweeks fantasy paranormal given to the pack,0.982
8979,the mating: the original law of the lycans story,nickycharles fantasy romance the mating: the o...,1.649
8980,my viking vampire,krystalshannan fantasy paranormal my viking va...,1.399


In [146]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [147]:
cvv = CountVectorizer()
text_vectorised = cvv.fit_transform(final_df['text']).toarray()

In [148]:
cos_sim = cosine_similarity(text_vectorised, text_vectorised)

In [152]:
def recommendation(book):
    all_per = []
    for books in final_df['book_title']:
        seq = difflib.SequenceMatcher(isjunk=None,a=book,b=books).ratio()  # This statement checks our main/single domain with the domains in the list
        per= round(seq*100, 2)
        all_per.append(per)
    
    final_query = final_df['book_title'].iloc[(pd.Series(all_per).sort_values(ascending=False)[:2].index)]
    idx = []
    for book in final_query:
        i = final_df[final_df['book_title'] == book ].index
        idx.append(i)
    final_recomm = [] 
    
    for i in idx:
        text_10 = pd.Series(cos_sim[i][0]).sort_values(ascending=False)[1:11]
#        print(text_10)
        text_idx = text_10.index
        text_recom_idx = (text_10 + final_df['numerical'].loc[text_idx]).index
        final_recomm.append(final_df['numerical'].loc[text_recom_idx])
    
    final_idx = pd.concat([final_recomm[0], final_recomm[1]]).sort_values(ascending=False).index
    final_recomm = final_df.iloc[final_idx]
    final_recomm = final_recomm[~final_recomm['book_title'].duplicated()]['book_title']
    
    print(final_recomm)

recommendation('murder on st.')

2850                        the poet
4322    murder on a girls' night out
3126               murder in chelsea
1067            a place of execution
0         murder on st. mark's place
656             absolution by murder
6752                         the son
8547                   the reckoning
4116                     the snowman
5649          still life with murder
5635    a plague on both your houses
8727                      the guards
4326                 on what grounds
4293             murder on the rocks
4279                   glazed murder
2847                         the bat
6777                hooked on murder
Name: book_title, dtype: object
